In [30]:
import os
from dotenv import load_dotenv
import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, Vector

load_dotenv()
# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") 
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") 
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") 
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") 
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT")
AZURE_SEARCH_SERVICE_KEY = os.environ.get("AZURE_SEARCH_SERVICE_KEY") 

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"


# Used by the OpenAI SDK
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-06-01-preview"
openai.api_key = os.environ.get("AZURE_OPENAI_API_KEY") 
# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = "azure" instead
openai.api_type = "azure"

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_SERVICE_KEY))

In [31]:
# ChatGPT uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
어시스턴트는 검색 결과에서 나온 문서에 대한 질문을 도와줍니다. 답변은 간결하게 작성하세요.
Source에 나오지 않은 내용은 모른다고 대답합니다. Source결과에 나온 내용을 잘 읽어보세요.
표 형식의 정보는 HTML 테이블로 반환합니다. 마크다운 형식은 반환하지 마세요.
각 출처에는 이름 뒤에 콜론과 실제 정보가 있으며, 응답에 사용하는 각 사실에 대한 출처 이름을 항상 포함하세요. 소스를 참조할 때는 대괄호를 사용합니다(예: [info1.txt]). 소스를 결합하지 말고 각 소스를 개별적으로 나열합니다(예: [info1.txt][info2.pdf]).
Sources:
{sources}

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """아래는 지금까지의 대화 요약과 사용자가 검색결과에서 검색하여 답변해야 하는 새로운 질문입니다. 대화와 새 질문을 기반으로 검색 쿼리를 생성합니다.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [32]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "인터넷 형광펜 서비스 소개좀 부탁해요"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

if len(history) > 0:
    completion = openai.Completion.create(
        engine=AZURE_OPENAI_GPT_DEPLOYMENT,
        prompt=summary_prompt_template.format(summary="\n".join(history), question=user_input),
        temperature=1,
        max_tokens=32,
        stop=["\n"])
    search = completion.choices[0].text
else:
    search = user_input

# Use Azure OpenAI to compute an embedding for the query
query_vector = openai.Embedding.create(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=search)["data"][0]["embedding"]


print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

# 1. only semantic search

# r = search_client.search(search, 
#                          filter=filter,
#                          query_type=QueryType.SEMANTIC, 
#                          query_language="en-us", 
#                          query_speller="lexicon", 
#                          semantic_configuration_name="default", 
#                          top=3)

# 2. Hybrid search with semantic search and vector reranking

# r = search_client.search(search, 
#                          filter=filter,
#                          query_type=QueryType.SEMANTIC, 
#                          query_language="en-us", 
#                          query_speller="lexicon", 
#                          semantic_configuration_name="default", 
#                          top=3,
#                          vector=query_vector, 
#                          top_k=50 if query_vector else None, 
#                          vector_fields="embedding" if query_vector else None)

# 3. Vector search only
r = search_client.search(search, top=3, vector=query_vector, top_k=50 if query_vector else None, vector_fields="embedding" if query_vector else None)

results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)

prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

print("\n-------------------\n".join(history))
print("\n-------------------\nPrompt:\n" + prompt)

completion = openai.Completion.create(
    engine=AZURE_OPENAI_CHATGPT_DEPLOYMENT, 
    prompt=prompt, 
    temperature=0,
    max_tokens=1024,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))




Searching: 인터넷 형광펜 서비스 소개좀 부탁해요
-------------------


-------------------
Prompt:
<|im_start|>system
어시스턴트는 검색 결과에서 나온 문서에 대한 질문을 도와줍니다. 답변은 간결하게 작성하세요.
Source에 나오지 않은 내용은 모른다고 대답합니다. Source결과에 나온 내용을 잘 읽어보세요.
표 형식의 정보는 HTML 테이블로 반환합니다. 마크다운 형식은 반환하지 마세요.
각 출처에는 이름 뒤에 콜론과 실제 정보가 있으며, 응답에 사용하는 각 사실에 대한 출처 이름을 항상 포함하세요. 소스를 참조할 때는 대괄호를 사용합니다(예: [info1.txt]). 소스를 결합하지 말고 각 소스를 개별적으로 나열합니다(예: [info1.txt][info2.pdf]).
Sources:


<|im_end|>
<|im_start|>user
인터넷 형광펜 서비스 소개좀 부탁해요
<|im_end|>
<|im_start|>assistant

user: 인터넷 형광펜 서비스 소개좀 부탁해요
-------------------
assistant: 인터넷 형광펜 서비스는 웹 브라우저 상에서 PDF나 웹페이지 등의 문서를 열어서 형광펜으로 강조하거나 메모를 추가할 수 있는 서비스입니다. 이를 통해 사용자는 문서를 보다 효율적으로 읽고, 필요한 부분을 쉽게 찾을 수 있습니다.

대표적인 인터넷 형광펜 서비스로는 Kami, Hypothesis, Notable PDF 등이 있습니다. 이들 서비스는 각각 다양한 기능을 제공하며, 무료 또는 유료로 이용할 수 있습니다.

Kami는 Google Drive와 연동되어 있어, Google Drive에 저장된 문서를 쉽게 열어서 형광펜으로 강조하거나 메모를 추가할 수 있습니다. Hypothesis는 학술 논문 등의 문서에 대한 협업 리뷰를 지원하며, Notable PDF는 형광펜 기능 뿐만 아니라 PDF 문서에 주석을 추가할 수 있는 기능도 제공합니다.

[출처: http

In [33]:
history.clear()